In [1]:
import os
import torch

from monai.data import  DataLoader
from monai.losses import DiceLoss
from torch.optim import Adam

from deeplabv3.network.modeling import _segm_resnet
from dataset import OcelotTissueDataset

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"device: {device}")

device: cpu


In [2]:
model = _segm_resnet(
    name="deeplabv3plus",
    backbone_name="resnet50",
    num_classes=2,
    output_stride=8,
    pretrained_backbone=True,
)
model.to(device)



DeepLabV3(
  (backbone): IntermediateLayerGetter(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Se

In [4]:
image_file_path = "ocelot_data/images/train/tissue/"
segmentation_file_path = "ocelot_data/annotations/train/tissue/"


image_files = [
    os.path.join(image_file_path, file_name)
    for file_name in os.listdir(image_file_path)
]
image_files.sort()

segmentation_files = [
    os.path.join(segmentation_file_path, file_name)
    for file_name in os.listdir(segmentation_file_path)
]
segmentation_files.sort()

In [5]:
dataset = OcelotTissueDataset(
    image_files=image_files, seg_files=segmentation_files, device=device
)

data_loader = DataLoader(dataset=dataset, batch_size=2)
loss_function = DiceLoss()
optimizer = Adam(model.parameters(), lr=1e-3)

/var/folders/kj/txkrf0nd1ws6sj4740kbsw300000gn/T/ipykernel_81982/1417595183.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  processed_images.append(torch.tensor(image).permute((2, 0, 1)))
/var/folders/kj/txkrf0nd1ws6sj4740kbsw300000gn/T/ipykernel_81982/1417595183.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  processed_labels.append(torch.tensor(label))


In [7]:
import time
num_epochs = 1
decode_fn = data_loader.dataset.decode_target

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    start = time.time()

    for (inputs, labels) in data_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        outputs = decode_fn(outputs).to(torch.float32)
        outputs.requires_grad = True

        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        print(f"One iteration")
    
    end = time.time()
    epoch_loss /= len(data_loader)
    print(f"Epoch {epoch + 1}, Loss: {epoch_loss}, epoch_time: {end-start:.2f} seconds")

One iteration
One iteration
One iteration


KeyboardInterrupt: 